# Spark Streaming 编程
---

## 概述

Spark Streaming是核心Spark API的扩展，可实现实时数据流的可扩展、高吞吐量、容错流处理。

Streaming框架如下：数据从多种数据源如Kafka、Kinesis、TCP套接字获取，经过高级别功能函数如`map`、`reduce`、`join`、`window`等表达的复杂算法处理,处理后的数据可推送到文件系统、数据库、实时仪表板。

![streaming-arch](../images/streaming-arch.png)

内部其工作方式如下：Spark Streaming接受实时输入数据流，并将数据流分批，然后由Spark引擎进行处理，最终生成批处理的结果流。

![streaming-flow](../images/streaming-flow.png)

Spark Streaming用称为离散流(DStream, discretized stream)的高级抽象来表示连续的数据流。DStream可根据来自Kafka和Kinesis等源的输入数据流来创建，也可通过对其他DStream应用高级操作来创建。在内部，DStream表示为RDD序列。

## Hello World

计算从侦听TCP套接字的数据服务器接收到的文本数据中的单词数。

In [1]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

# 创建具有两个执行线程和1秒批处理间隔的本地StreamingContext
sc = SparkContext("local[2]", "NetworkWordCount")
ssc = StreamingContext(sc, 1)

In [2]:
# 使用此上下文创建一个DStream，表示来自TCP源的流数据
lines = ssc.socketTextStream("localhost", 9999)

In [3]:
# DStream中每个记录都是一行文本，将其分成单子列表
words = lines.flatMap(lambda line: line.split(" "))

In [4]:
pairs = words.map(lambda word: (word, 1))
wordCounts = pairs.reduceByKey(lambda x, y: x + y)

wordCounts.pprint()

在执行下列代码前，在另一终端开启netcat服务器 `nc -lk 9999` 并输入hello world，待启动下列代码后再回车。

In [5]:
# 设置好DStream启动计算
ssc.start()
import time
time.sleep(5)
ssc.stop()

-------------------------------------------
Time: 2020-09-15 13:19:12
-------------------------------------------

-------------------------------------------
Time: 2020-09-15 13:19:13
-------------------------------------------

-------------------------------------------
Time: 2020-09-15 13:19:14
-------------------------------------------

-------------------------------------------
Time: 2020-09-15 13:19:15
-------------------------------------------

-------------------------------------------
Time: 2020-09-15 13:19:16
-------------------------------------------
('world', 1)
('hello', 1)

-------------------------------------------
Time: 2020-09-15 13:19:17
-------------------------------------------



## 基本概念

### 流上下文

**流上下文(StreamingContext)**是所有Spark Streaming函数的主要入口点，可从SparkContext对象创建。

```python
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

sc = SparkContext(master, appName)
ssc = StreamingContext(sc, 1)
```

定义上下文后，必须执行以下操作。

1. 通过创建输入DStream定义输入源。
2. 通过将转换和输出操作应用于DStream来定义流计算。
3. 开始接收数据并使用进行处理`streamingContext.start()`。
4. 等待使用停止处理（手动或由于任何错误）`streamingContext.awaitTermination()`。
5. 可以使用手动停止处理`streamingContext.stop()`。

**要点**：
- 一旦启动上下文，就无法设置新的流计算或将其添加到该流计算中。
- 上下文一旦停止，就无法重新启动。
- JVM中只能同时激活一个StreamingContext。
- StreamingContext上的stop()也会停止SparkContext。要仅停止的StreamingContext，设置`stop()`的可选参数stopSparkContext为false。
- 只要在创建下一个StreamingContext之前停止（而不停止SparkContext）上一个StreamingContext，即可将SparkContext重用于创建多个StreamingContext。

### 流

**流(DStream)**是Spark Streaming提供的基本抽象，它表示连续的数据流。在内部，DStream由一系列连续的RDD表示，每个RDD都包含特定间隔的数据，如下图：

![streaming-dstream](../images/streaming-dstream.png)

在DStream上执行的任何操作都转换为对基础RDD的操作,如下图：

![streaming-dstream-ops](../images/streaming-dstream-ops.png)

基础的RDD转换由Spark引擎计算。


### 输入流和接收器

输入流(Input DStream)是表示从流源接受输入数据的DStream，在[Hello World](#hello-world)例子中，lines即为输入流，表示从netcat服务器接收的数据流。每个输入流（文件流除外）都与一个**接收器(Receiver)**关联，接收器从源接收数据并将其存储在Spark的内存中进行处理。

Spark Streaming提供了两类内置的流媒体源。
- 基础源：可直接在StreamingContext API中获得的源。如：文件系统和套接字连接
- 高级源：可以通过其他实用程序类如Kafka，Kinesis等获得资源。

流应用程序中可创建过个输入流及接收器，可同时接收多个数据流。

**要点**
- 在本地运行流应用程序时，请勿使用`"local"`或`"local[1]"`作为Master URL。这两种方式均意味着仅一个线程将用于本地运行任务。如果使用了关联接收器（例如套接字，Kafka等）的输入流，则将使用单个线程来运行接收器，而不会留下任何线程来处理接收到的数据。因此，在本地运行时，请始终使用`"local[n]"`作为Master URL，其中n>要运行的接收器数。
- 同理，在集群上运行流应用程序，分配给流应用程序的内核数必须大于接收器数。否则，系统能接收数据，但无法处理它。

#### 基础源

**基于TCP套接字创建流**: ` ssc.socketTextStream(...)`。在例子[Hello World](#hello-world)中已展示。

**基于文件系统创建流**：`streamingContext.textFileStream(dataDirectory)`。读取任何兼容HDFS API(如HDFS、S3、NFS等)的文件系统中的文件来创建DStream。文件系统不需要运行接收器，因此不需要分配任何内核来接受文件数据。若输入为目录，Spark Streaming将监视目录并处理目录中创建的所有文件。

**基于RDD队列创建流**：`streamingContext.queueStream(queueOfRDDs)`。

****

#### 高级源
从Spark 3.0.1开始，Python API中提供了Kafka和Kinesis源。

- Kafka： Spark Streaming 3.0.1与0.10或更高版本的Kafka代理兼容。有关更多详细信息，请参见《[Kafka集成指南](https://spark.apache.org/docs/latest/streaming-kafka-0-10-integration.html)》。
- Kinesis： Spark Streaming 3.0.1与Kinesis Client Library 1.2.1兼容。有关更多详细信息，请参见《[Kinesis集成指南](https://spark.apache.org/docs/latest/streaming-kinesis-integration.html)》。

#### 自定义源
Python API尚不支持。

#### 接收器的可靠性
根据是否会发送反馈，接收器分为

- **可靠接收器**:接受到数据并存储到Spark中后，会向源发送acknowledgement

- **不可靠接收器**:不会向源发送acknowledgement

### 流的转换

转换|含义
:--|:--
**map**(*func*)|返回一个*func*计算结果的新DStream
**flatMap**(*func*) | 类似于map，但是每一个输入元素可以被映射为0或多个输出元素（所以*func*应该返回一个序列，而不是单一元素）
**filter**(*func*) | 返回一个挑选*func*计算结果为true的元素的新DStream
**repartition**(*numPartitions*) | 重新分区，必须shuffle，参数是要分多少区
**union**(*otherDStream*) | 返回源DStream和*otherDStream*的并集DStream
**count**() | 返回一个单元素RDD的DStream，统计每个RDD的元素个数
**reduce**(*func*) | 类似count，不过这里指定*func*作为”加“算子
**countByValue**() | 返回（K，V）对的DStream，V为键K在每个RDD中的频率
**reduceByKey**(*func*,\[*numTasks*\]) | 类似countByValue，不过这里指定*func*作为”加“算子
**join**(*otherStream*, \[*numTasks*\]) | 在(K,V)和(K,W)的DStream上调用，返回(K,(V,W))的DStream，其中每个键都有所有成对的元素，相当于内连接(求交集)。
**cogroup**(*otherStream*, \[*numTasks*\]) | 在(K,V)和(K,W)的DStream上调用，返回一个`(K, Seq[V],Seq[W])`类型的DStream。
**transform**(*func*) | 执行任意的RDD-to-RDD的函数
**updateStateByKey**(*func*) | 返回一个更新状态的DStream， 每个键对应的状态通过在该键先前状态和当前状态应用给定函数*func*来更新


### 窗口操作

Spark Streaming提供窗口计算，允许用户对滑动窗口数据进行转换，如下图：

![streaming-dstream-window](../images/streaming-dstream-window.png)

任何窗口操作都需要指定两个参数：

- windowLength: 窗口长度
- slideInterval: 进行窗口操作的间隔

如下列代码实现每10s在统计一次最后30s数据的字数

In [ ]:
windowedWordCounts = pairs.reduceByKeyAndWindow(lambda x, y: x + y, lambda x, y: x - y, 30, 10)

以下为常见的窗口操作

转换 | 含义
:-- | :--
**window**(*windowLength*, *slideInterval*) | 返回基于源DStream的窗口批处理计算的新DStream。
**countByWindow**(*windowLength*, *slideInterval*) | 	返回流中元素的滑动窗口计数。
**reduceByWindow**(*func*, *windowLength*, *slideInterval*) | 返回一个新的单元素流，该流是通过使用func在滑动间隔内聚合流中的元素而创建的。 该函数应该是关联的和可交换的，以便可以并行正确地计算它。
**reduceByKeyAndWindow**(*func*, *windowLength*, *slideInterval*, \[*numTasks*\]) | 当在（K，V）对的DStream上调用时，返回一个新的（K，V）对的DStream，其中每个键的值使用给定的reduce函数func在滑动窗口的批处理中聚合。 注意：默认情况下，它使用Spark的默认并行任务数（本地模式为2，而在集群模式下，此数量由config属性spark.default.parallelism确定）进行分组。 您可以传递一个可选的numTasks参数来设置不同数量的任务。
**reduceByKeyAndWindow**(*func*, *invfunc*, *windowLength*, *slideInterval*, \[*numTasks*\]) | 上述reduceByKeyAndWindow（）的更有效的版本，其中，使用前一个窗口的缩减值递增地计算每个窗口的缩减值。 这是通过减少进入滑动窗口的新数据，然后“反减少”离开窗口的旧数据来完成的。 一个示例是在窗口滑动时“增加”和“减少”键的计数。 但是，它仅适用于“可逆归约函数”，即具有相应“逆归约”函数（作为参数invFunc的归约）的归约函数。 像reduceByKeyAndWindow中一样，reduce任务的数量可以通过可选参数配置。 请注意，必须启用检查点才能使用此操作。
**countByValueAndWindow**(*windowLength*, *slideInterval*, \[*numTasks*\]) | 在（K，V）对的DStream上调用时，返回新的（K，Long）对的DStream，其中每个键的值是其在滑动窗口内的频率。 像reduceByKeyAndWindow中一样，reduce任务的数量可以通过可选参数配置。

### 流的输出操作

### DataFrame和SQL操作

### MLlib操作

### 缓存与持久化

### 检查点

### 累加器、广播变量和检查点

### 部署流应用

### 监视流应用

## 性能调优

### 减少批处理时间

### 设置正确的批间隔

### 内存调优

## 容错语义